## This notebook was only created to test the ReadAndRunAllWorkFlows.py


<br>
<br>

In [163]:
import requests
import json
from collections import defaultdict
import os
import pandas as pd

In [2]:
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io/legacy'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

In [58]:
def retrieve_ars_results(mid,ars_url='https://ars.transltr.io/ars/api'):
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    #print( j['status'] )
    results = {}
    dictionary = {}
    for child in j['children']:
        #print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
        else:
            nresults = 0
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results'
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error'
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error'
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results'
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return dictionary


def submit_to_devars(m):
    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

def retrieve_devars_results(m):
     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))

In [27]:
urls = ['/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/EGFR_advanced.json', '/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.1_RHOBTB2.json']

In [31]:
dict_workflows = {}
for i in urls:
    feature = (os.path.splitext(os.path.basename(i))[0])
    print(i, feature)
    with open(i,'r') as inf:
        query = json.load(inf)
        kcresult = submit_to_devars(query)
        #xx = retrieve_devars_results(kcresult)
        
        dict_workflows[feature] = kcresult

/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/EGFR_advanced.json EGFR_advanced
https://arax.ncats.io/?source=ARS&id=9da061af-0a64-443e-8f92-324200b87546
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.1_RHOBTB2.json A.1_RHOBTB2
https://arax.ncats.io/?source=ARS&id=3067c839-3bdc-4923-82d4-9a50c8e3b36e


In [33]:
dict_workflows

{'EGFR_advanced': '9da061af-0a64-443e-8f92-324200b87546',
 'A.1_RHOBTB2': '3067c839-3bdc-4923-82d4-9a50c8e3b36e'}

In [59]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    print(keys, val)
    
    result_status = retrieve_devars_results(val)
    
    workflow_result_messages[keys] = result_status

EGFR_advanced 9da061af-0a64-443e-8f92-324200b87546
ara-aragorn Error 0
ara-arax Error 0
ara-bte Error 0
ara-unsecret Error 0
kp-genetics Done 0
kp-molecular Error 0
ara-explanatory Done 0
ara-improving ARS Error 0
kp-cam Done 0
kp-textmining Error 0
kp-openpredict Done 0
kp-cohd Error 0
kp-chp Done 0
kp-icees Done 0
kp-icees-dili Done 0
A.1_RHOBTB2 3067c839-3bdc-4923-82d4-9a50c8e3b36e
ara-aragorn Done 6
ara-arax Done 2
ara-bte Error 0
ara-unsecret Error 0
kp-genetics Done 0
kp-molecular Error 0
ara-explanatory Done 0
ara-improving Done 4
kp-cam Done 0
kp-textmining Error 0
kp-cohd Error 0
kp-openpredict Done 0
kp-chp Done 0
kp-icees Done 0
kp-icees-dili Done 0


In [60]:
workflow_result_messages

{'EGFR_advanced': {'ara-aragorn': 'Error',
  'ara-arax': 'Error',
  'ara-bte': 'Error',
  'ara-unsecret': 'Error',
  'kp-genetics': 'No Results',
  'kp-molecular': 'Error',
  'ara-explanatory': 'No Results',
  'ara-improving': 'ARS Error',
  'kp-cam': 'No Results',
  'kp-textmining': 'Error',
  'kp-openpredict': 'No Results',
  'kp-cohd': 'Error',
  'kp-chp': 'No Results',
  'kp-icees': 'No Results',
  'kp-icees-dili': 'No Results'},
 'A.1_RHOBTB2': {'ara-aragorn': 'Results',
  'ara-arax': 'Results',
  'ara-bte': 'Error',
  'ara-unsecret': 'Error',
  'kp-genetics': 'No Results',
  'kp-molecular': 'Error',
  'ara-explanatory': 'No Results',
  'ara-improving': 'Results',
  'kp-cam': 'No Results',
  'kp-textmining': 'Error',
  'kp-cohd': 'Error',
  'kp-openpredict': 'No Results',
  'kp-chp': 'No Results',
  'kp-icees': 'No Results',
  'kp-icees-dili': 'No Results'}}

In [169]:
col = []
final_dict = defaultdict(list)
for k in sorted(workflow_result_messages):
    col.append(k)
    
    for key, value in workflow_result_messages[k].items():
            final_dict[key].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

In [171]:
df

,A.1_RHOBTB2,EGFR_advanced
ara-aragorn,Results,Error
ara-arax,Results,Error
ara-bte,Error,Error
ara-unsecret,Error,Error
kp-genetics,No Results,No Results
kp-molecular,Error,Error
ara-explanatory,No Results,No Results
ara-improving,Results,ARS Error
kp-cam,No Results,No Results
kp-textmining,Error,Error


In [167]:
df.to_excel('test_ara_worklow.xlsx', sheet_name = 'Progress_Tracker')

In [168]:
pwd

'/Users/priyash/Documents/GitHub/minihackathons/Notebooks'

In [172]:
## test for individual run

In [175]:
with open('/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.1_RHOBTB2.json','r') as inf:
    query1 = json.load(inf)

In [177]:
kcresult = submit_to_devars(query1)

https://arax.ncats.io/?source=ARS&id=b46a4258-c405-46ad-9494-fbbfa90cfdae


In [179]:
result_status = retrieve_devars_results(kcresult)

ara-aragorn Running 0
ara-arax Done 9
ara-bte Done 8
ara-unsecret Done 0
kp-genetics Error 0
kp-molecular Done 0
ara-explanatory Done 0
ara-improving Done 4
kp-cam Done 0
kp-textmining Done 0
kp-openpredict Running 0
